In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [113]:
# Fucntions:
def convert(x):
    if x>0:
        return x
    else:
        return 0

In [3]:
# My results:
path="/Users/Chelo/Documents/UCL/Term2/WebEconomics/project/results_validation2.csv"
res1=pd.read_csv(path,low_memory=False, dtype={'bid_id': object})
res1=res1.drop('Unnamed: 0',axis=1)
res1=res1.rename(columns={"pCTR":"pCTR_chelo","bidprice":"bidprice_chelo"})
res1.loc[259686,"bid_id"]='5.13E+31'

In [4]:
# Ulyses results:
res2=pd.read_csv("/Users/Chelo/Downloads/csv_result.csv")
res2=res2.drop('Unnamed: 0',axis=1)
res2=res2.rename(columns={"pCTR":"pCTR_uly"})

In [96]:
# Shafik results:
res3=pd.read_csv("/Users/Chelo/Downloads/shafik_val.csv")
res3=res3.drop(['Unnamed: 0',"click"],axis=1)
res3.bidprice_shaf=res3.bidprice_shaf*1000
res3.bidprice_shaf=res3.apply(lambda x: convert(x.bidprice_shaf),axis=1)
res3=res3.rename(columns={"pCTR":"pCTR_shaf"})

In [6]:
total_res = res2.merge(res1, on=['bid_id'],how="left").merge(res3,on=["bid_id"])

In [7]:
total_res=total_res.drop(["advertiser_x","advertiser_y"],axis=1)
total_res=total_res.rename(columns={"click_x":"click","payprice_x":"payprice"})
total_res=total_res.drop(["payprice_y"],axis=1)

In [9]:
del(res1)
del(res2)
del(res3)

In [222]:
def strategy(bid1,bid2,bid3,pCTR1,pCTR2,pCTR3,strategy="mean"):
    if strategy=="max":
        final_bid = max(bid1,bid2,bid3)
    if strategy=="mean":
        final_bid = (bid1+bid2+bid3)/3
    if strategy=="min":
        final_bid = min(bid1,bid2,bid3) 
    if strategy == "agreement_max_min":
        mean_pCTR=np.mean([pCTR1,pCTR2,pCTR3])
        if mean_pCTR>0.5:
            final_bid=max(bid1,bid2,bid3)
        else:
            final_bid=min(bid1,bid2,bid3)
    if strategy == "agreement_mean_0":
        mean_pCTR=np.mean([pCTR1,pCTR2,pCTR3])
        if mean_pCTR>0.5:
            final_bid=np.mean([bid1,bid2,bid3])
        else:
            final_bid=0
    if strategy == "crazy_agreement":
        mean_pCTR=np.mean([pCTR1,pCTR2,pCTR3])
        if mean_pCTR>=0.75:
            final_bid=max(bid1,bid2,bid3)
        elif mean_pCTR>0.5 and mean_pCTR<0.75:
            final_bid=np.mean([bid1,bid2,bid3])
        else:
            final_bid=0
    return final_bid

In [223]:
total_res.head(1)

,bid_id,bidprice_uly,pCTR_uly,bidprice_chelo,payprice,pCTR_chelo,click,advertiser,bidprice_shaf,pCTR_shaf
0,91c6a6b9e90c0f54d3230815a5a3e22e,0.0,0.347201,0.028161,102,0.000029,0,3427,0.0,0.148061


In [18]:

def RTB_simulation(total_res,group_strategy="mean"):
    print "start simulation"
    k=0
    impressions=0
    clicks=0
    spend=0
    budget=6250000

    results=[]
    for row in total_res.iterrows():
        # Values:
        values=row[1]
        # Bids:
        bid1=int(values[1])
        bid2=int(values[3])
        bid3=int(values[8])
        # pCTrs:
        pCTR1=float(values[2])
        pCTR2=float(values[5])
        pCTR3=float(values[9])
        
        payprice=values[4]
        click_true=int(values[6])

        # apply strategy
        final_bid=strategy(bid1,bid2,bid3,pCTR1,pCTR2,pCTR3,strategy=group_strategy)

        if final_bid<budget:
            k+=1
            if final_bid>=payprice:
                # Won bid:
                impressions+=1
                budget-=payprice
                spend+=payprice
                if click_true==1:
                    # There was a click
                    clicks+=1
    if impressions>0:
        pCTR=(clicks/impressions)*100
    else:
        pCTR=-1
    results.append((group_strategy,clicks,impressions,pCTR,spend,k))
    print results
    return results

In [224]:
results_mean=RTB_simulation(total_res,group_strategy="mean")
results_max=RTB_simulation(total_res,group_strategy="max")
results_min=RTB_simulation(total_res,group_strategy="min")

start simulation
[('mean', 144, 65807, 0.2188217058975489, 3314893, 299749)]
start simulation
[('max', 139, 89921, 0.1545801314487161, 6249999, 256837)]
start simulation
[('min', 35, 7019, 0.49864653084484967, 276147, 299749)]


In [225]:
results_agg=RTB_simulation(total_res,group_strategy="agreement_max_min")

start simulation
[('agreement_max_min', 153, 49334, 0.31013094417643006, 4251979, 299749)]


In [226]:
results_agg_mean=RTB_simulation(total_res,group_strategy="agreement_mean_0")

start simulation
[('agreement_mean_0', 132, 37342, 0.3534893685394462, 2300620, 299749)]


In [227]:
results_crazy=RTB_simulation(total_res,group_strategy="crazy_agreement")

start simulation
[('crazy_agreement', 134, 37460, 0.3577148958889482, 2324947, 299749)]


In [236]:
val_results=[]

In [237]:
val_results=results_mean+results_max+results_min+results_agg+results_agg_mean+results_crazy

In [240]:
val_results=pd.DataFrame(final_results,columns=["Strategy","clicks","impressions","CTR","spend","stop row"])

In [241]:
val_results.to_csv("validation_res.csv")

,Strategy,clicks,impressions,CTR,spend,stop row
0,mean,144,65807,0.218822,3314893,299749
1,max,139,89921,0.154580,6249999,256837
2,min,35,7019,0.498647,276147,299749
3,agreement_max_min,153,49334,0.310131,4251979,299749
4,agreement_mean_0,132,37342,0.353489,2300620,299749
5,crazy_agreement,134,37460,0.357715,2324947,299749


## TEst set:

In [202]:
test_path_1="/Users/Chelo/Documents/UCL/Term2/WebEconomics/project/test_results_chelo.csv"

In [203]:
test1=pd.read_csv(test_path_1,low_memory=False, dtype={'bid_id': str})
test1=test1.drop('Unnamed: 0',axis=1)
test1.bidprice=test1.bidprice.astype(int)
test1=test1.rename(columns={"pCTR":"pCTR_chelo","bidprice":"bidprice_chelo","bid_id":"bidid"})


In [204]:
# Shafik results:
test3=pd.read_csv("/Users/Chelo/Downloads/shafik_test.csv")
test3=test3.drop(['Unnamed: 0',"PayPrice","advertiser"],axis=1)
test3=test3.rename(columns={"myProb":"pCTR_shaf","myBid":"bidprice_shaf","bid_id":"bidid"})
test3.bidprice_shaf=test3.apply(lambda x: convert(x.bidprice_shaf),axis=1)
test3.bidprice_shaf=test3.bidprice_shaf*1000

In [205]:
# Ulyses results:
test2=pd.read_csv("/Users/Chelo/Downloads/ulyses_test.csv",low_memory=False)
test2=test2.drop(['0',"advertiser"],axis=1)
test2=test2.rename(columns={"pCTR":"pCTR_uly","bidprice":"bidprice_uly"})

In [206]:
total_test = test2.merge(test1, on=['bidid'],how="left").merge(test3,on=["bidid"])

In [207]:
total_test.head(1)

,bidid,bidprice_uly,pCTR_uly,bidprice_chelo,pCTR_chelo,bidprice_shaf,pCTR_shaf
0,fe2e06dff1dfd227471fd1ca717888ac,0.0,0.403661,150,0.35131,0.0,0.635743


In [233]:

def RTB_simulation_test(total_test,group_strategy="mean"):
    print "start simulation"
    k=0
    impressions=0
    spend=0
    budget=6250000

    results=[]
    for row in total_res.iterrows():
        # Values:
        values=row[1]
        bidid=values[0]
        # Bids:
        bid1=int(values[1])
        bid2=int(values[3])
        bid3=int(values[5])
        # pCTrs:
        pCTR1=float(values[2])
        pCTR2=float(values[4])
        pCTR3=float(values[6])

        # apply strategy
        final_bid=strategy(bid1,bid2,bid3,pCTR1,pCTR2,pCTR3,strategy=group_strategy)
        
        #results.append((bidid,np.mean([bid1,bid2,bid3]),final_bid))
        results.append((bidid,final_bid))
    return results

In [234]:
test_results=RTB_simulation_test(total_test,group_strategy="agreement_max_min")

start simulation


In [246]:
test_df=pd.DataFrame(test_results,columns=["bidid","bidprice"])

In [249]:
test_df.to_csv("testing_bidding_price.csv")